###Count number of processes to bring exact number of records

In [ ]:
import requests as r

url = "https://api.funcionjudicial.gob.ec/EXPEL-CONSULTA-CAUSAS-SERVICE/api/consulta-causas/informacion/contarCausas"
payload = {
    "actor": {"cedulaActor": "0968599020001", "nombreActor": ""},
    "demandado": {"cedulaDemandado": "", "nombreDemandado": ""},
    "recaptcha": "verdad"
}
number_process = r.post(url, json=payload)
print(number_process.text)

157


###Get processes with the identity document of actor or defendant.

In [ ]:
import requests as r
import json

url = "https://api.funcionjudicial.gob.ec/EXPEL-CONSULTA-CAUSAS-SERVICE/api/consulta-causas/informacion/buscarCausas?page=1&size=1"
payload = {
    "actor": {"cedulaActor": "0968599020001", "nombreActor": ""},
    "demandado": {"cedulaDemandado": "", "nombreDemandado": ""},
    "recaptcha": "verdad",
}
response = r.post(url, json=payload)
all_process = json.dumps(response.json(), indent=2)
print(all_process)

[
  {
    "id": 1,
    "idJuicio": "08332202400176",
    "estadoActual": "A",
    "idMateria": 32,
    "idProvincia": null,
    "idCanton": null,
    "idJudicatura": null,
    "nombreDelito": "NULIDAD DE SENTENCIA",
    "fechaIngreso": "2024-03-06T21:13:25.237+00:00",
    "nombre": null,
    "cedula": null,
    "idEstadoJuicio": null,
    "nombreMateria": null,
    "nombreEstadoJuicio": null,
    "nombreJudicatura": null,
    "nombreTipoResolucion": null,
    "nombreTipoAccion": null,
    "fechaProvidencia": null,
    "nombreProvidencia": null,
    "nombreProvincia": null,
    "iedocumentoAdjunto": "N"
  }
]


###Get detail processes
At this point you can distinguish between defendant and plaintiff processes.

In [ ]:
import requests as r

url = "https://api.funcionjudicial.gob.ec/EXPEL-CONSULTA-CAUSAS-CLEX-SERVICE/api/consulta-causas-clex/informacion/getIncidenteJudicatura/08332202400176"
headers = {"Content-Type": "application/json"}
response = r.get(url, headers=headers)
detail_process = json.dumps(response.json(), indent=2)
print(detail_process)


[
  {
    "idJudicatura": "08332",
    "nombreJudicatura": "UNIDAD JUDICIAL MULTICOMPETENTE CIVIL DE QUININD\u00c9",
    "ciudad": "QUININDE",
    "lstIncidenteJudicatura": [
      {
        "idIncidenteJudicatura": 27389734,
        "idMovimientoJuicioIncidente": 26015981,
        "idJudicaturaDestino": "08332",
        "fechaCrea": "2024-03-06T16:13:25.237",
        "incidente": 1,
        "lstLitiganteActor": [
          {
            "tipoLitigante": "ACTOR",
            "nombresLitigante": "EMPRESA ELECTRICA PUBLICA ESTRATEGICA CORPORACION NACIONAL DE ELECTRICIDAD CNEL EP",
            "representadoPor": "EMPRESA ELECTRICA PUBLICA ESTRATEGICA CORPORACION NACIONAL DE ELECTRICIDAD CNEL EP",
            "idLitigante": 35888938
          }
        ],
        "lstLitiganteDemandado": null,
        "litiganteActor": null,
        "litiganteDemandado": null
      }
    ]
  }
]


###Obtain the judicial updates according to the Judiciary Incident Data

In [ ]:
import requests as r

url = "https://api.funcionjudicial.gob.ec/EXPEL-CONSULTA-CAUSAS-SERVICE/api/consulta-causas/informacion/actuacionesJudiciales"
payload = {
    "idMovimientoJuicioIncidente": 26015981,
    "idJuicio": "08332202400176",
    "idJudicatura": "08332",
    "idIncidenteJudicatura": 27389734,
    "aplicativo": "web",
    "nombreJudicatura": "UNIDAD JUDICIAL MULTICOMPETENTE CIVIL DE QUININDÉ",
    "incidente": 1
}
response = r.post(url, json=payload)
judicial_update = json.dumps(response.json(), indent=2)
print(judicial_update)

[
  {
    "codigo": 211954195,
    "idJudicatura": "08332",
    "idJuicio": "08332202400176 ",
    "fecha": "2024-03-13T20:23:39.000+00:00",
    "tipo": "OFICIO (OFICIO) ",
    "actividad": "<p style=\"margin-left:0cm; margin-right:0cm; text-align:justify\">Dentro del Juicio de <strong>NULIDAD DE SENTENCIA </strong>que sigue <strong>EMPRESA ELECTRICA PUBLICA ESTRATEGICA CORPORACION NACIONAL DE ELECTRICIDAD CNEL EP </strong>se ha dispuesto, <strong>mediante auto de fecha, 8 de marzo del 2024, a las 08h47</strong>, lo siguiente:</p>\n\n<p style=\"margin-left:0cm; margin-right:0cm; text-align:justify\"><strong>PARTE PERTINENTE: </strong>&ldquo;&hellip;conforme obra de la documentaci&oacute;n adjuntada al libelo inicial, con fundamento en los Arts. 22.4 y 112 pen&uacute;ltimo inciso del C&oacute;digo Org&aacute;nico General de Procesos, me EXCUSO de conocer el presente juicio ordinario de nulidad de sentencia, excusa que la presento para ante el/la Juez/a al que, por sorteo le corresponda,

In [ ]:
# GET ALL PROCESS and DETAILS and actuaciones judiciales

# Crear y documentar un caso de prueba dentro del cual se ejecuten 15 consultas paralelas revisando que no haya un bloqueo por parte de la página de consulta.

import requests as r
import csv
import json

BASE_URL = "https://api.funcionjudicial.gob.ec"

url_processes= BASE_URL + "/EXPEL-CONSULTA-CAUSAS-SERVICE/api/consulta-causas/informacion/buscarCausas?page=1&size=157"
url_act = BASE_URL + "/EXPEL-CONSULTA-CAUSAS-SERVICE/api/consulta-causas/informacion/actuacionesJudiciales"

payload = {
    "actor": {"cedulaActor": "0968599020001", "nombreActor": ""},
    "demandado": {"cedulaDemandado": "", "nombreDemandado": ""},
    "recaptcha": "verdad",
}
list_data = []

response = r.post(url_processes, json=payload)
all_processes = response.json()

for process in all_processes:
  data = {}
  if payload["actor"]["cedulaActor"]:
    process["type_process"] = "demandante"
  else:
    process["type_process"] = "demandado"
  data = process
  data["id_process"] = data.pop("id")
  url = f"{BASE_URL}/consulta-causas-clex/informacion/getIncidenteJudicatura/{process['idJuicio']}"
  all_details = r.get(f"{BASE_URL}/EXPEL-CONSULTA-CAUSAS-CLEX-SERVICE/api/consulta-causas-clex/informacion/getIncidenteJudicatura/{process['idJuicio']}")
  details = all_details.json()
  data["details"] = details

  if details:
    for detail in details:
      payload_judicial_updates = {
          "aplicativo": "web"
      }
      payload_judicial_updates["idJuicio"] = process["idJuicio"]
      payload_judicial_updates["idJudicatura"] = detail["idJudicatura"]
      payload_judicial_updates["nombreJudicatura"] = detail["nombreJudicatura"]
      for incidente in detail["lstIncidenteJudicatura"]:
        payload_judicial_updates["idIncidenteJudicatura"] = incidente["idIncidenteJudicatura"]
        payload_judicial_updates["idMovimientoJuicioIncidente"] = incidente["idMovimientoJuicioIncidente"]
        payload_judicial_updates["incidente"] = incidente["incidente"]
        all_judicial_updates = r.post(url_act, json=payload_judicial_updates)
        a = all_judicial_updates.json()
        data["judicial_updates"] = a
  list_data.append(data)

#print(json.dumps(list_data, indent=2))
file_path = "/content/data.json"

# Escribir la data en el archivo JSON
with open(file_path, 'w') as file:
    json.dump(list_data, file)

print("Data exportada exitosamente como JSON.")


Data exportada exitosamente como JSON.


###Caso de prueba

In [ ]:
import requests as r
import csv
import json

BASE_URL = "https://api.funcionjudicial.gob.ec"

url_processes= BASE_URL + "/EXPEL-CONSULTA-CAUSAS-SERVICE/api/consulta-causas/informacion/buscarCausas?page=1&size=157"
url_act = BASE_URL + "/EXPEL-CONSULTA-CAUSAS-SERVICE/api/consulta-causas/informacion/actuacionesJudiciales"

payload = {
    "actor": {"cedulaActor": "0968599020001", "nombreActor": ""},
    "demandado": {"cedulaDemandado": "", "nombreDemandado": ""},
    "recaptcha": "verdad",
}
list_data = []

response = r.post(url_processes, json=payload)
all_processes = response.json()

for process in all_processes:
  data = {}
  if payload["actor"]["cedulaActor"]:
    process["type_process"] = "demandante"
  else:
    process["type_process"] = "demandado"
  data = process
  data["id_process"] = data.pop("id")
  url = f"{BASE_URL}/consulta-causas-clex/informacion/getIncidenteJudicatura/{process['idJuicio']}"
  all_details = r.get(f"{BASE_URL}/EXPEL-CONSULTA-CAUSAS-CLEX-SERVICE/api/consulta-causas-clex/informacion/getIncidenteJudicatura/{process['idJuicio']}")
  details = all_details.json()
  data["details"] = details

  if details:
    for detail in details:
      payload_judicial_updates = {
          "aplicativo": "web"
      }
      payload_judicial_updates["idJuicio"] = process["idJuicio"]
      payload_judicial_updates["idJudicatura"] = detail["idJudicatura"]
      payload_judicial_updates["nombreJudicatura"] = detail["nombreJudicatura"]
      for incidente in detail["lstIncidenteJudicatura"]:
        payload_judicial_updates["idIncidenteJudicatura"] = incidente["idIncidenteJudicatura"]
        payload_judicial_updates["idMovimientoJuicioIncidente"] = incidente["idMovimientoJuicioIncidente"]
        payload_judicial_updates["incidente"] = incidente["incidente"]
        all_judicial_updates = r.post(url_act, json=payload_judicial_updates)
        a = all_judicial_updates.json()
        data["judicial_updates"] = a
  list_data.append(data)

#print(json.dumps(list_data, indent=2))
file_path = "/content/data.json"

# Escribir la data en el archivo JSON
with open(file_path, 'w') as file:
    json.dump(list_data, file)

print("Data exportada exitosamente como JSON.")